<a href="https://colab.research.google.com/github/MiguelDiLalla/Media_Generators/blob/main/Colab_AudioProcesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: install dependencies

!pip install whisper


In [1]:
import os
import json
import numpy as np
import librosa
import whisper
from google.colab import files

# Function to upload an audio file
def upload_audio():
    print("Please upload your audio file:")
    uploaded = files.upload()
    if len(uploaded) > 0:
        audio_path = list(uploaded.keys())[0]
        print(f"[INFO] Uploaded: {audio_path}")
        return audio_path
    else:
        raise Exception("No file uploaded.")

# Function to compute audio intensity
def compute_audio_intensity(audio_path):
    print("[INFO] Computing audio intensity...")
    y, sr = librosa.load(audio_path, sr=22050)  # Load audio and downsample
    rms = librosa.feature.rms(y=y, frame_length=2048, hop_length=512)[0]  # RMS energy
    normalized_rms = rms / np.max(rms)  # Normalize to [0, 1]
    print("[INFO] Audio intensity computation complete.")
    return normalized_rms, sr

# Function to transcribe audio and compute WPM
def compute_wpm(audio_path):
    print("[INFO] Transcribing audio with Whisper...")
    model = whisper.load_model("base")  # Use Whisper's base model for faster processing
    result = model.transcribe(audio_path)
    text = result['text']

    # Compute words per minute (WPM)
    word_count = len(text.split())
    duration_minutes = result['duration'] / 60  # Duration in minutes
    wpm = word_count / duration_minutes

    print("[INFO] Transcription and WPM calculation complete.")
    return text, wpm

# Function to save results as JSON
def save_results(intensity, wpm, output_path="audio_data.json"):
    print(f"[INFO] Saving results to {output_path}...")
    data = {
        "intensity": intensity.tolist(),  # Convert NumPy array to list
        "wpm": wpm
    }
    with open(output_path, "w") as f:
        json.dump(data, f)
    print(f"[INFO] Results saved to {output_path}.")
    return output_path

# Main execution flow
def main():
    # Step 1: Upload audio file
    audio_path = upload_audio()

    # Step 2: Process audio
    intensity, sr = compute_audio_intensity(audio_path)

    # Step 3: Transcribe audio and compute WPM
    text, wpm = compute_wpm(audio_path)

    # Step 4: Save results
    output_file = save_results(intensity, wpm)

    # Step 5: Download results
    print("[INFO] Preparing results for download...")
    files.download(output_file)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'whisper'